In [1]:
""" Script to take planning applications and match to OSAddressBase addresses

Written by: Christine Langston, March 2024
"""
import pandas as pd
import time
import numpy as np
import copy
import re

In [2]:
#create a function to iterate through csv. use for OSAddressBase file
def read_csv(file_name, columns):
    for chunk in pd.read_csv(file_name, chunksize=10000, usecols=columns, 
                            dtype={'urpn':str, 'parent_urpn': str, 'class': str, 'latitude': float, 
                                                  'longitude': float, 'country': str}):
    
        #if chunk['country'] == 'E':
        yield chunk


In [3]:
#takes a merged dataset and separates matched addresses and not matched
def separate_matches(dataset, column_name, match_strategy):
    dataset_match = dataset.drop(dataset[pd.isna(dataset[column_name]) == True].index)
    dataset_no_match = dataset.drop(dataset[pd.isna(dataset[column_name]) == False].index)
    dataset_match['match_strategy'] = match_strategy
    return dataset_match, dataset_no_match

In [4]:
#function to merge and then split based on if an address was matched  
def my_merge(left, right, left_on, right_on): 
    merged = left.merge(right, how = 'left', left_on = left_on, right_on = right_on)
    merged_match = merged.drop(merged[pd.isna(merged['uprn']) == True].index) 
    merged_no_match = merged.drop(merged[pd.isna(merged['uprn']) == False].index) 
    return merged_match, merged_no_match

In [5]:
# Read in the data
"""
fewer_columns  = ['uprn', 'parent_uprn', 'class', 'latitude', 'longitude', 'country' ]
all_columns = ['uprn', 'parent_uprn', 'class', 'latitude', 'longitude', 'country', 'legal_name','sub_building_name',
           'building_name','building_number','sao_start_number','sao_start_suffix','sao_end_number',
           'sao_end_suffix','sao_text','alt_language_sao_text','pao_start_number','pao_start_suffix',
           'pao_end_number','pao_end_suffix','pao_text','alt_language_pao_text','usrn','usrn_match_indicator',
           'area_name','level','official_flag','os_address_toid','os_address_toid_version','os_roadlink_toid',
           'os_roadlink_toid_version','os_topo_toid','os_topo_toid_version','voa_ct_record','voa_ndr_record',
           'street_description','alt_language_street_description','dependent_thoroughfare','thoroughfare',
           'double_dependent_locality','dependent_locality','locality', 'town_name', 
          'administrative_area','post_town','postcode','postcode_locator' ]
"""

address_matching_columns = ['uprn', 'parent_uprn', 'class', 'latitude', 'longitude', 'country',
                    'legal_name','sub_building_name', 'building_name','building_number','street_description',
                    'dependent_locality','locality', 'town_name', 'administrative_area','post_town','postcode', 'postcode_locator' ]

address_base_file = "/Users/christine/Documents/_UCL_grad school/research/ab_plus_england_202308150944.csv"

#nana_process_file = "/Users/christine/OneDrive - University College London/WP1_Address list compilation/13_stage_3_from_NanaWei/PA_batch1_London_202402_processed.csv"

nana_process_file = "/Users/christine/Documents/_UCL_grad school/research/Batch2_London_filtered_with_manual&Glenigan.csv"


process_file_ben_columns = '/Users/christine/OneDrive - University College London/WP1_Address list compilation/13_stage_3_from_NanaWei/PA_batch1 with units and type_03192024.csv'



In [6]:
#london_file = "/Users/christine/OneDrive - University College London/Attachments/PDdata_barch1_London_filtered.csv"
london_data = pd.read_csv(nana_process_file)

In [7]:
#### DATA EXPLORATION - SKIP IF JUST MATCHING 
#there are 1338 addresses in the London file, 1063 have a URPN, 275 do not have UPRN
#1338 unique properties, 907 unique uprn
london_data['ID'].nunique()



2966

In [8]:
london_data['uprn'].nunique()

461

In [9]:
london_data.head()

,ID,planning_application_number,lpa_name,application_type,application_type_full,description,number_of_units,site_number_clean,street_name,postcode_clean,site_name_clean,site_name_GLA,site_name_LPA,uprn,decision,status,application_date,decision_date
0,1543,15/P0673,Merton,Prior Approval,Prior Approval (Class M - formerly IA),PRIOR APPROVAL IN RESPECT OF THE PROPOSED CHAN...,1.0,209,Manor Road,_x000D__x000D_\nCR4 1JH,NaN,"209, Manor Road, CR4 1JH",209 Manor Road_x000D__x000D_\nMitcham _x000D__...,NaN,Prior Approval Granted,Completed,02/03/2015,16/04/2015
1,1544,15/P0298,Merton,Prior Approval,Prior Approval (Class O - formerly J),Prior approval in relation to the change of us...,1.0,21 & 21A,Willow Lane,_x000D__x000D_\nCR4 4NA,Clock House & Connect House,"Connect House, 21a And Clock House, 21, Willow...",Clock House_x000D__x000D_\n21 Willow Lane &_x0...,NaN,Prior Approval Granted,Completed,09/02/2015,29/03/2015
2,1544,15/P0298,Merton,Prior Approval,Prior Approval (Class O - formerly J),Prior approval in relation to the change of us...,51.0,21 & 21A,Willow Lane,_x000D__x000D_\nCR4 4NA,Clock House & Connect House,"Connect House, 21a And Clock House, 21, Willow...",Clock House_x000D__x000D_\n21 Willow Lane &_x0...,NaN,Prior Approval Granted,Completed,09/02/2015,29/03/2015
3,1544,15/P0298,Merton,Prior Approval,Prior Approval (Class O - formerly J),Prior approval in relation to the change of us...,26.0,21 & 21A,Willow Lane,_x000D__x000D_\nCR4 4NA,Clock House & Connect House,"Connect House, 21a And Clock House, 21, Willow...",Clock House_x000D__x000D_\n21 Willow Lane &_x0...,NaN,Prior Approval Granted,Completed,09/02/2015,29/03/2015
4,1544,15/P0404,Merton,Prior Approval,Prior Approval (Class O - formerly J),Prior approval in relation to the change of us...,5.0,21 & 21A,Willow Lane,_x000D__x000D_\nCR4 4NA,Clock House & Connect House,"Clock House And Connect House, 21-21a, Willow ...",Clock House_x000D__x000D_\n21 Willow Lane &_x0...,NaN,Prior Approval Granted,Completed,09/02/2015,01/04/2015


In [10]:
#### DATA EXPLORATION - SKIP IF JUST MATCHING 
#how many rows without uprns have an address? 
#london_data_no_uprn = london_data.dropna(subset=["uprn
london_data_no_uprn = london_data.drop(london_data[pd.isna(london_data['uprn'])==True].index)

In [11]:
london_data_no_uprn.nunique()

ID                             513
planning_application_number    628
lpa_name                         8
application_type                 1
application_type_full           17
description                    596
number_of_units                 45
site_number_clean              304
street_name                    287
postcode_clean                 405
site_name_clean                120
site_name_GLA                  476
site_name_LPA                  494
uprn                           461
decision                        24
status                          10
application_date               465
decision_date                  503
dtype: int64

In [12]:
#read in the address base data - 32 Million rows
t = time.time()
df_lst = [] 


# Iterate over the file based on the criteria and append to the list
for df in read_csv(address_base_file, address_matching_columns):
       # if df['country'] == 'E':
       # tmp_df = (df.pipe(lambda x:  x[x.country == 'E'] ))
    df_lst +=   [df.copy()] 

# And finally combine filtered df_lst into the final larger output say 'df_final' dataframe 
df_final = pd.concat(df_lst)
print('seconds: ', time.time() - t)

/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_1373/257141313.py:3: DtypeWarning: Columns (21,22,56,62,64) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_name, chunksize=10000, usecols=columns,
/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_1373/257141313.py:3: DtypeWarning: Columns (21,22,56,62,64) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_name, chunksize=10000, usecols=columns,
/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_1373/257141313.py:3: DtypeWarning: Columns (21,22,56,62,64) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(file_name, chunksize=10000, usecols=columns,
/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_1373/257141313.py:3: DtypeWarning: Columns (56) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_c

seconds:  247.27735805511475


In [13]:
### ------ DATA CLEANING  ----------
#edit the parent uprn from OSAddress base to fill out to be strings with 12 digits and leading zeros
#replace nan
df_final = df_final.replace([np.nan, -np.inf], 0)

#cast as integer
df_final['parent_uprn'] = df_final['parent_uprn'].astype('Int64')
df_final['uprn'] = df_final['uprn'].astype('Int64')


In [14]:
### ------ DATA CLEANING  ----------
#cast as string 
df_final['parent_uprn'] = df_final['parent_uprn'].astype(str)
df_final['uprn'] = df_final['uprn'].astype(str)

#fill in with left side padding zeros 
df_final['parent_uprn'] = df_final['parent_uprn'].apply(lambda x: '{0:0>12}'.format(x))
df_final['uprn'] = df_final['uprn'].apply(lambda x: '{0:0>12}'.format(x))

#print(df_final['parent_uprn'])

In [18]:
### ------ DATA CLEANING  ----------

#BATCH 1 london_data has extra quotation marks, need to remove
#london_data['uprn'] = london_data['uprn'].apply(lambda x: x.strip("''") if not pd.isna(x) else x)


In [16]:
### ------ DATA CLEANING  ----------
london_data['uprn'] = london_data['uprn'].astype('Int64')



In [17]:
#cast the strings and make sure it looks good 
london_data['uprn'] = london_data['uprn'].astype('str') #apply(lambda x: str(x) if not pd.isna(x) else x)

london_data['uprn'] = london_data['uprn'].apply(lambda x: '{0:0>12}'.format(x) if not pd.isna(x) else x)

In [19]:
### ------ DATA CLEANING  ----------

london_data = london_data.replace('00000000<NA>',np.NaN)

In [20]:
### ------ DATA CLEANING  ----------
#street address from site_name_LPA
london_data['parsed_street_LPA'] = london_data['site_name_LPA'].apply(lambda x: re.findall("[0-9]+.-?.[0-9]+?\s(.+)(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens|Drive)", x) if not pd.isna(x) and '-' in x
                                                                  else (re.findall("[0-9]+\s(.+)(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens|Drive)", x) if not pd.isna(x) else []))


In [21]:
london_data['parsed_street_LPA'] =  london_data['parsed_street_LPA'].apply(lambda x: x[0][0] + x[0][1]  if len(x) > 0 else None)

In [22]:
#cleaning - street address
london_data['parsed_street_GLA'] = london_data['site_name_GLA'].apply(lambda x: re.findall("[0-9]+.-?.[0-9]+?\s(.+)(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens|Drive)", x) if not pd.isna(x) and '-' in x
                                                                  else (re.findall("[0-9]+\s(.+)(Road|Lane|Avenue|Parade|Courtyard|Street|Gardens|Drive)", x) if not pd.isna(x) else []))


In [23]:
london_data['parsed_street_GLA'] =  london_data['parsed_street_GLA'].apply(lambda x: x[0][0] + x[0][1]  if len(x) > 0 else None)

In [24]:
#if the original street was empty, then we want to use the LPA Or GLA parsed name as street_name 

london_data['street_name'] = np.where(london_data['street_name'].isnull(), np.where(london_data['parsed_street_LPA'].isnull(), london_data['parsed_street_GLA'], london_data['parsed_street_LPA']), london_data['street_name'] )

In [26]:
#make london_data all into capitals? 
london_data['street_name'] = london_data['street_name'].str.upper()

In [27]:
# DATA EXPLORATION - EXPORT CSV IF NEEDED 
london_data.to_csv('batch2_cleaned.csv', index = False)

In [28]:
### ------ DATA CLEANING  ----------

#Address matching data clean, make building number into a string 
df_final['building_number'] = df_final['building_number'].astype('Int64').astype('str') 

In [29]:
#split the AddressBase into Residential uses and all others
# 26,492,127 data ponits
resi_AB = df_final[df_final['class'].str.startswith('R')] 
other_AB = df_final[df_final['class'].str.startswith('R') == False] 

In [30]:
# DATA EXPLORATION - EXPORT CSV IF NEEDED 
resi_AB.to_csv('resi_AB_cleaned.csv', index = False)

In [31]:
# DATA EXPLORATION - EXPORT CSV IF NEEDED 
other_AB.to_csv('other_AB_cleaned.csv', index = False)

In [ ]:
#### READ IN THE DATA IF EXPORTED 
resi_AB = pd.read_csv('resi_AB_cleaned.csv')
other_AB = pd.read_csv('other_AB_cleaned.csv')
london_data = pd.read_csv('batch2_cleaned.csv')

In [96]:
#df_final[df_final['postcode'] == 'BR1 1JH']

,uprn,class,parent_uprn,latitude,longitude,country,legal_name,sub_building_name,building_name,building_number,street_description,dependent_locality,locality,town_name,administrative_area,post_town,postcode
19513590,100022886877,CR01,000000000000,51.404505,0.015341,E,0.0,0,0,143,HIGH STREET,0,0,BROMLEY,BROMLEY,BROMLEY,BR1 1JH


In [100]:
#london_data[london_data['postcode_clean'] == 'BR1 1JH']

,ID,planning_application_number,lpa_name,application_type,application_type_full,description,number_of_units,site_number_clean,street_name,postcode_clean,site_name_clean,site_name_GLA,site_name_LPA,uprn,decision,status,application_date,decision_date,parsed_street_LPA,parsed_street_GLA
26,1561,16/05471/RESPA,Bromley,Prior Approval,Prior Approval (Class O - formerly J),Change of use of third and fourth floor office...,4.0,143,HIGH STREET,BR1 1JH,NaN,"143, High Street, BR1 1JH",143 High Street Bromley BR1 1JH,NaN,Grant Prior Approval,Lapsed,09/12/2016,23/01/2017,High Street,None
27,1561,16/05471/RESPA,Bromley,Prior Approval,Prior Approval (Class O - formerly J),Change of use of third and fourth floor office...,4.0,143,HIGH STREET,BR1 1JH,NaN,"143, High Street, BR1 1JH",143 High Street Bromley BR1 1JH,NaN,Grant Prior Approval,Lapsed,09/12/2016,23/01/2017,High Street,None
28,1561,17/00490/RESPA,Bromley,Prior Approval,Prior Approval (Class O - formerly J),Change of use of second floor offices from Cla...,2.0,143,HIGH STREET,BR1 1JH,NaN,"143, High Street, BR1 1JH",143 High Street Bromley BR1 1JH,NaN,Grant Prior Approval,Lapsed,07/02/2017,23/02/2017,High Street,None
29,1561,17/00490/RESPA,Bromley,Prior Approval,Prior Approval (Class O - formerly J),Change of use of second floor offices from Cla...,2.0,143,HIGH STREET,BR1 1JH,NaN,"143, High Street, BR1 1JH",143 High Street Bromley BR1 1JH,NaN,Grant Prior Approval,Lapsed,07/02/2017,23/02/2017,High Street,None


In [32]:
#### -------- DATA MERGING --------
#join the london data with the RESIDENTIAL AddressBase dataset on UPRN
merged = london_data.merge(resi_AB, how = 'left', left_on = 'uprn', right_on = 'parent_uprn')

#merged['UCL_ID'].nunique()

In [33]:
#separate merged into no match and match 
merged_match, merged_no_match = separate_matches(merged, 'parent_uprn', 'parent_uprn')

merged_no_match = merged_no_match.dropna(axis=1, how='all')

In [34]:
#merge 2 on uprn not parent_uprn 
merged_2 = merged_no_match.merge(resi_AB, how = 'left', left_on = 'uprn_x', right_on = 'uprn')

In [35]:
#separate the merge 2 into two datasets for match v not match 
merged_2_match, merged_2_no_match = separate_matches(merged_2, 'uprn', 'uprn')

merged_2_no_match = merged_2_no_match.dropna(axis=1, how='all')      

In [36]:
merged_2_match = merged_2_match.rename(columns={"uprn": "uprn_OSAB"})
merged_match = merged_match.rename(columns={"uprn_y": "uprn_OSAB"})

#merged_2_match.count()

In [37]:
all_matched = pd.concat([merged_2_match, merged_match])


In [107]:
#-------- DATA MERGING -------- 
## ADDRESS STRATEGY ONE 
#match on the street number, street name, postcode 
left_columns = ['site_number_clean', 'street_name', 'postcode_clean']
right_columns = ['building_number', 'street_description', 'postcode']

# this is a very strict conservative join 
merged_on_address = merged_2_no_match.merge(resi_AB, how = 'left', left_on = left_columns, right_on = right_columns)

In [108]:
merged_on_address.count()

ID                             14381
planning_application_number    14381
lpa_name                       14381
application_type               14381
application_type_full          14381
description                    14379
number_of_units                14233
site_number_clean              13181
street_name                    14294
postcode_clean                 13813
site_name_clean                 7709
site_name_GLA                  12850
site_name_LPA                  13813
uprn_x                           791
decision                       12623
status                         14381
application_date               10719
decision_date                  14379
parsed_street_LPA               8205
parsed_street_GLA                277
uprn                            9954
class                           9954
parent_uprn                     9954
latitude                        9954
longitude                       9954
country                         9954
legal_name                      9954
s

In [95]:
#int64
#london_data[london_data['ID'] == 1561]
print(merged_on_address[merged_on_address['ID'] == 1561])

#non_resi_address_merge_no_match3[non_resi_address_merge_no_match3['ID'] == 1561]

      ID planning_application_number lpa_name application_type  \
26  1561              16/05471/RESPA  Bromley   Prior Approval   
27  1561              16/05471/RESPA  Bromley   Prior Approval   
28  1561              17/00490/RESPA  Bromley   Prior Approval   
29  1561              17/00490/RESPA  Bromley   Prior Approval   

                    application_type_full  \
26  Prior Approval (Class O - formerly J)   
27  Prior Approval (Class O - formerly J)   
28  Prior Approval (Class O - formerly J)   
29  Prior Approval (Class O - formerly J)   

                                          description  number_of_units  \
26  Change of use of third and fourth floor office...              4.0   
27  Change of use of third and fourth floor office...              4.0   
28  Change of use of second floor offices from Cla...              2.0   
29  Change of use of second floor offices from Cla...              2.0   

   site_number_clean  street_name postcode_clean  ... sub_building_name 

In [109]:
merged_on_address_match,merged_on_address_no_match =  separate_matches(merged_on_address, 'uprn', 'address_1')


In [110]:
## Add the merged_on_address_match to the matched
merged_on_address_match = merged_on_address_match.rename(columns={"uprn": "uprn_OSAB"})
#merged_on_address_match['match_strategy'] = 'address_1'


In [111]:
all_matched = pd.concat([all_matched, merged_on_address_match])


In [112]:
# we would want to create a new column on this dataset that cleans up the site name
merged_on_address_no_match['short_site_name_LPA]' ] = np.where(merged_on_address_no_match['site_name_LPA'].isnull() == False ,
                                                        merged_on_address_no_match['site_name_LPA'].apply(lambda x: x.split(',')[0] if not pd.isna(x) else x ).str.lower(), 
                                                        merged_on_address_no_match['site_name_GLA'].apply(lambda x: x.split('\r\r\n')[0] if not pd.isna(x) else x ).str.lower()  )





In [113]:
merged_on_address_no_match = merged_on_address_no_match.dropna(axis=1, how='all')


In [114]:
##-------- DATA MERGING --------  STRATEGY 2 ADDRESS MATCH
# concat AB Building number + AB street description = site_name_LPA before comma
    #  and postcode  = postcode 
#create new column and oncatenate the buildign number and street

resi_AB['number_street'] = np.where(resi_AB['building_name'] != 0,  resi_AB['building_name'].apply(lambda x: re.sub('[A-Z]', '',x) if type(x) == str else x)
, resi_AB['building_number'])

/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_2689/3348808103.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resi_AB['number_street'] = np.where(resi_AB['building_name'] != 0,  resi_AB['building_name'].apply(lambda x: re.sub('[A-Z]', '',x) if type(x) == str else x)


In [115]:
resi_AB['number_street'] =  resi_AB['number_street'] + ' ' + resi_AB['street_description'].str.lower()

/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_2689/2416929871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resi_AB['number_street'] =  resi_AB['number_street'] + ' ' + resi_AB['street_description'].str.lower()


In [116]:
left_columns_2 = ['short_site_name_LPA]', 'postcode_clean'] 
right_columns_2 = ['number_street', 'postcode']

merged_on_address2 = merged_on_address_no_match.merge(resi_AB, how = 'left', left_on = left_columns_2, right_on = right_columns_2)

In [117]:
merged_on_address2_match = merged_on_address2.drop(merged_on_address2[pd.isna(merged_on_address2['uprn']) == True].index) 
merged_on_address2_no_match = merged_on_address2.drop(merged_on_address2[pd.isna(merged_on_address2['uprn']) == False].index) 


In [118]:
merged_on_address2_match = merged_on_address2_match.rename(columns={"uprn": "uprn_OSAB"})
merged_on_address2_match = merged_on_address2_match.drop(columns = ['short_site_name_LPA]']) 
merged_on_address2_match['match_strategy'] = 'address_2'

In [119]:
all_matched = pd.concat([all_matched, merged_on_address2_match])

In [120]:
merged_on_address2_no_match = merged_on_address2_no_match.dropna(axis=1, how='all')


In [121]:
# #-------- DATA MERGING --------  STRATEGY 3 ADDRESS MATCH 
# concat AB Building number + AB street description = site_name_LPA before comma
    # And AB administrative_area = LPA_name
left_columns_3 = ['short_site_name_LPA]', 'lpa_name'] 
right_columns_3 = ['number_street', 'administrative_area']

merged_on_address2_no_match['lpa_name'] = merged_on_address2_no_match['lpa_name'].str.upper()
#merged_on_address3 = merged_on_address2_no_match.merge(resi_AB, how = 'left', left_on = left_columns_3, right_on = right_columns_3)

merged_on_address3_match, merged_on_address3_no_match = my_merge(merged_on_address2_no_match, resi_AB, left_columns_3, right_columns_3)

In [122]:
# add the matched into all matched
merged_on_address3_match = merged_on_address3_match.rename(columns={"uprn": "uprn_OSAB"})
merged_on_address3_match = merged_on_address3_match.drop(columns = ['short_site_name_LPA]']) 
merged_on_address3_match['match_strategy'] = 'address_3'

frames = [all_matched, merged_on_address3_match]

all_matched = pd.concat(frames)

In [123]:
merged_on_address3_no_match = merged_on_address3_no_match.dropna(axis=1, how='all')

In [112]:
#columns_to_drop = ['legal_name','sub_building_name','building_name','building_number','street_description','dependent_locality','locality','town_name','administrative_area','post_town','postcode','uprn','class','parent_uprn','latitude','longitude','country', 'number_street_y']
#merged_on_address3_no_match = merged_on_address3_no_match.drop(columns=columns_to_drop)

In [ ]:
#  STRATEGY 4 ADDRESS MATCH 
#match based on address number and postcode
#left_columns_4 = ['site_number_clean', 'postcode_clean']
#right_columns_4 = ['building_number', 'postcode']

# this is a loose join 
#merged_on_address4 = merged_on_address3_no_match.merge(resi_AB, how = 'left', left_on = left_columns_4, right_on = right_columns_4)

#other ideas: 
# if site name starts with a non number, remove the first comma and try strat 2 


In [125]:
##-------- DATA MERGING --------  COMMERCIAL PROPERTIES --- Reproduce the process with commercial properties 
#Merge on Parent UPRN 
non_resi_merged = merged_on_address3_no_match.merge(other_AB,how = 'left', left_on = 'uprn_x', right_on = 'parent_uprn')

non_resi_match, non_resi_no_match =  separate_matches(non_resi_merged, 'parent_uprn', 'parent_uprn')

non_resi_no_match = non_resi_no_match.dropna(axis=1, how='all')

In [126]:
#merge on UPRN
non_resi_merged_2 = non_resi_no_match.merge(other_AB, how = 'left', left_on = 'uprn_x', right_on = 'uprn')

non_resi_match_2, non_resi_no_match_2 =  separate_matches(non_resi_merged_2, 'parent_uprn', 'uprn')

non_resi_no_match_2 = non_resi_no_match_2.dropna(axis=1, how='all')

In [127]:
non_resi_match_2 = non_resi_match_2.rename(columns={"uprn": "uprn_OSAB"})
#non_resi_match_2['match_strategy'] = 'uprn'
non_resi_match_2.count()

non_resi_match = non_resi_match.rename(columns={"uprn": "uprn_OSAB"})
#non_resi_match['match_strategy'] = 'parent_uprn'

In [128]:
#union them together 
non_resi_all_matched = pd.concat([non_resi_match_2, non_resi_match])

In [129]:
#now join on addresses strat 1 
left_columns = ['site_number_clean', 'street_name', 'postcode_clean']
right_columns = ['building_number', 'street_description', 'postcode']

non_resi_address_merge_match, non_resi_address_merge_no_match = my_merge(non_resi_no_match_2, other_AB, left_columns, right_columns)

In [130]:
non_resi_address_merge_no_match = non_resi_address_merge_no_match.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match = non_resi_address_merge_match.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match['match_strategy'] = 'address_1'

non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match])

In [131]:
#join addresses strat 2
#make change to the other_AB dataset 
other_AB['number_street'] = np.where(other_AB['building_name'] != 0,  other_AB['building_name'].apply(lambda x: re.sub('[A-Z]', '',x) if type(x) == str else x)
, other_AB['building_number'])
other_AB['number_street'] =  other_AB['number_street'] + ' ' + other_AB['street_description'].str.lower()


left_columns_2 = ['short_site_name_LPA]', 'postcode_clean'] 
right_columns_2 = ['number_street', 'postcode']
 
non_resi_address_merge_match2, non_resi_address_merge_no_match2 = my_merge(non_resi_address_merge_no_match, other_AB, left_columns_2,right_columns_2 )

/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_2689/3097699056.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_AB['number_street'] = np.where(other_AB['building_name'] != 0,  other_AB['building_name'].apply(lambda x: re.sub('[A-Z]', '',x) if type(x) == str else x)
/var/folders/0b/k156jw2x3rz7y1k26v1wkh880000gn/T/ipykernel_2689/3097699056.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_AB['number_street'] =  other_AB['number_street'] + ' ' + other_AB['street_description'].str.

In [132]:
non_resi_address_merge_no_match2 = non_resi_address_merge_no_match2.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match2 = non_resi_address_merge_match2.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match2['match_strategy'] = 'address_2'

non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match2])

In [133]:
# STRATEGY 3 ADDRESS MATCH 
# concat AB Building number + AB street description = site_name_LPA before comma
    # And AB administrative_area = LPA_name
left_columns_3 = ['short_site_name_LPA]', 'lpa_name'] 
right_columns_3 = ['number_street', 'administrative_area']

non_resi_address_merge_no_match2['lpa_name'] = non_resi_address_merge_no_match2['lpa_name'].str.upper()
#merged_on_address3 = merged_on_address2_no_match.merge(resi_AB, how = 'left', left_on = left_columns_3, right_on = right_columns_3)

non_resi_address_merge_match3, non_resi_address_merge_no_match3 = my_merge(non_resi_address_merge_no_match2, other_AB, left_columns_3, right_columns_3)

In [134]:
non_resi_address_merge_no_match3 = non_resi_address_merge_no_match3.dropna(axis=1, how='all')

## Add the merged_on_address_match to the matched
non_resi_address_merge_match3 = non_resi_address_merge_match3.rename(columns={"uprn": "uprn_OSAB"})
non_resi_address_merge_match3['match_strategy'] = 'address_3'

non_resi_all_matched = pd.concat([non_resi_all_matched, non_resi_address_merge_match3])

In [71]:
## ------------------------------- Post Match  -------------------------------

In [135]:
#print match rate
resi_match_rate = 100 * all_matched['ID'].nunique() / london_data['ID'].nunique()

print('Resi Match rate: ', resi_match_rate) 

Resi Match rate:  24.578556979096426


In [136]:
other_match_rate = 100 * non_resi_all_matched['ID'].nunique() / london_data['ID'].nunique()

print('Other Match rate: ', other_match_rate) 

Other Match rate:  22.791638570465274


In [137]:
#before export, make all these columns blank instead of 0 
#'legal_name', 'sub_building_name', 'building_name','building_number','street_description', 'dependent_locality	locality
all_matched['street_name'] = all_matched['street_name'].replace(0,'')
all_matched['legal_name'] = all_matched['legal_name'].replace(0,'')
all_matched['sub_building_name'] = all_matched['sub_building_name'].replace(0,'')
all_matched['building_name'] = all_matched['building_name'].replace(0,'')
all_matched['building_number'] = all_matched['building_number'].replace(0,'')
all_matched['street_description'] = all_matched['street_description'].replace(0,'')
all_matched['dependent_locality'] = all_matched['dependent_locality'].replace(0,'')
all_matched['locality'] = all_matched['locality'].replace(0,'')
all_matched['post_town'] = all_matched['post_town'].replace(0,'')
all_matched['postcode'] = np.where(all_matched['postcode'] == 0, all_matched['postcode_clean'], all_matched['postcode'])


In [76]:
all_matched.head(20)

,UCL_ID,planning_application_number,lpa_name,application_type,application_type_full,description,number_of_units,site_number_clean,street_name,postcode_clean,...,town_name,administrative_area,post_town,postcode,match_strategy,Number_units_found,FPP_PA_mix?,number_street,number_street_x,number_street_y
42,54,21/01316/PREZA,Newham,Prior Approval,Prior Approval: Change of use - light industri...,Application to determine if prior approval is ...,NaN,2A,Boundary Road,E13 9PR,...,LONDON,NEWHAM,LONDON,E13 9PR,uprn,NaN,NaN,NaN,NaN,NaN
87,123,21/00771/PRECOU,Newham,Prior Approval,Prior Approval: Change of use - retail/service...,Prior approval for the change of the retail un...,3.0,140,Portway,E15 3QW,...,LONDON,NEWHAM,LONDON,E15 3QW,uprn,NaN,NaN,NaN,NaN,NaN
93,132,19/AP/1141,Southwark,Prior Approval,Prior Approval: Change of use - storage to dwe...,Notification for prior approval for a change o...,4.0,4A,None,SE1 4QG,...,LONDON,SOUTHWARK,LONDON,SE1 4QG,uprn,NaN,NaN,NaN,NaN,NaN
99,142,22/02610/PRECOU,Newham,Prior Approval,"Prior Approval: Change of use from Commercial,...",Prior approval for the change of use of the ba...,NaN,140,Portway,E15 3QW,...,LONDON,NEWHAM,LONDON,E15 3QW,uprn,NaN,NaN,NaN,NaN,NaN
102,145,22/AP/3633,Southwark,Prior Approval,"Prior Approval: Change of use from Commercial,...",Prior approval notification for the change of ...,NaN,3,ONEGA GATE,SE16 7PF,...,LONDON,SOUTHWARK,,SE16 7PF,uprn,NaN,NaN,NaN,NaN,NaN
106,149,22/03970/PIAPA,Westminster,Prior Approval,"Prior Approval: Change of use from Commercial,...",Application for Prior Approval Under Class G o...,NaN,54,Rochester Row,SW1P 1JU,...,LONDON,CITY OF WESTMINSTER,LONDON,SW1P 1JU,uprn,NaN,NaN,NaN,NaN,NaN
115,163,20/2001/PNO,Barnet,Prior Approval,NaN,Change of use of ground and lower ground floor...,1.0,3,Leicester Road,EN5 5EW,...,BARNET,BARNET,BARNET,EN5 5EW,uprn,NaN,NaN,NaN,NaN,NaN
118,167,22/1285/GPD26,Richmond,Prior Approval,NaN,CHANGE OF USE TO SINGLE DWELLING HOUSE,1.0,5,Bridle Lane,TW1 3EG,...,TWICKENHAM,RICHMOND UPON THAMES,TWICKENHAM,TW1 3EG,uprn,NaN,NaN,NaN,NaN,NaN
130,181,21/03383/PRIOR,Bexley,Prior Approval,NaN,Notification for Prior Approval for change of ...,1.0,13,Sidcup High Street,DA14 6EP,...,SIDCUP,BEXLEY,SIDCUP,DA14 6EP,uprn,NaN,NaN,NaN,NaN,NaN
134,185,DM2020/00970,Sutton,Prior Approval,NaN,Prior Approval for change of use from an offic...,1.0,177,Gander Green Lane,SM1 2EZ,...,SUTTON,SUTTON,SUTTON,SM1 2EZ,uprn,NaN,NaN,NaN,NaN,NaN


In [140]:
#write out to csv 

all_matched.to_csv('London2_address_matched_2603.csv', index=False) 

non_resi_all_matched.to_csv('London2_non_resi_match_2603.csv', index=False)  

In [141]:
non_resi_address_merge_no_match3.to_csv('London2_no_match_2603.csv', index = False)

In [86]:
non_resi_address_merge_no_match3[non_resi_address_merge_no_match3['ID'] == 1561]

,ID,planning_application_number,lpa_name,application_type,application_type_full,description,number_of_units,site_number_clean,street_name,postcode_clean,...,site_name_GLA,site_name_LPA,uprn_x,decision,status,application_date,decision_date,parsed_street_LPA,parsed_street_GLA,short_site_name_LPA]
24,1561,16/05471/RESPA,BROMLEY,Prior Approval,Prior Approval (Class O - formerly J),Change of use of third and fourth floor office...,4.0,143,High Street,BR1 1JH,...,"143, High Street, BR1 1JH",143 High Street Bromley BR1 1JH,NaN,Grant Prior Approval,Lapsed,09/12/2016,23/01/2017,High Street,None,143 high street bromley br1 1jh
25,1561,16/05471/RESPA,BROMLEY,Prior Approval,Prior Approval (Class O - formerly J),Change of use of third and fourth floor office...,4.0,143,High Street,BR1 1JH,...,"143, High Street, BR1 1JH",143 High Street Bromley BR1 1JH,NaN,Grant Prior Approval,Lapsed,09/12/2016,23/01/2017,High Street,None,143 high street bromley br1 1jh
26,1561,17/00490/RESPA,BROMLEY,Prior Approval,Prior Approval (Class O - formerly J),Change of use of second floor offices from Cla...,2.0,143,High Street,BR1 1JH,...,"143, High Street, BR1 1JH",143 High Street Bromley BR1 1JH,NaN,Grant Prior Approval,Lapsed,07/02/2017,23/02/2017,High Street,None,143 high street bromley br1 1jh
27,1561,17/00490/RESPA,BROMLEY,Prior Approval,Prior Approval (Class O - formerly J),Change of use of second floor offices from Cla...,2.0,143,High Street,BR1 1JH,...,"143, High Street, BR1 1JH",143 High Street Bromley BR1 1JH,NaN,Grant Prior Approval,Lapsed,07/02/2017,23/02/2017,High Street,None,143 high street bromley br1 1jh
